In [ ]:
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import mnist

In [1]:
def calculate_centroid(image, row_start, col_start):
    # Calculate the sum of pixels and weighted sums for x and y
    sum_pixels = np.sum(image)
    if sum_pixels == 0:
        # If sum_pixels is zero, all pixels in the block are black, return 0 for centroid
        return 0, 0

    num_rows, num_cols = image.shape

    sum_weighted_x = 0
    sum_weighted_y = 0

    for i in range(num_rows):
        for j in range(num_cols):
            sum_weighted_x += (i + row_start) * image[i, j]
            sum_weighted_y += (j + col_start) * image[i, j]

    # Calculate centroids
    centroid_x = sum_weighted_x / sum_pixels
    centroid_y = sum_weighted_y / sum_pixels

    return centroid_x, centroid_y


In [2]:
def extract_features(images, n_blocks):
    features = []
    for image in images:
        block_size = image.shape[0] // int(np.sqrt(n_blocks))  # Assuming the image is square
        block_features = []
        for i_block in range(int(np.sqrt(n_blocks))):
            for j_block in range(int(np.sqrt(n_blocks))):
                row_start = i_block * block_size
                col_start = j_block * block_size
                row_end = row_start + block_size
                col_end = col_start + block_size
                block = image[row_start:row_end, col_start:col_end]
                centroid_x, centroid_y = calculate_centroid(block, row_start, col_start)
                block_features.extend([centroid_x, centroid_y])
        features.append(block_features)
    return np.array(features)


In [3]:
# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# Normalize pixel values to the range [0, 1]
X_train = X_train / 255.0
X_test = X_test / 255.0


11490434/11490434 [==============================] - 1s 0us/step


In [4]:
# Extract features using centroid-based approach
n_blocks = 9
X_train_features = extract_features(X_train, n_blocks)
X_test_features = extract_features(X_test, n_blocks)


In [8]:
# Train SVM classifier
svm_classifier = SVC(kernel='poly')
svm_classifier.fit(X_train_features, y_train)

# Make predictions
y_pred = svm_classifier.predict(X_test_features)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8986
